In [13]:
import json
import numpy as np
# this reference implementation solves the challenge in ~1h using 16 threads on an 8-core machine with 8Gb RAM
# it produces the submission.txt file that we share for demo purposes.

from tqdm import tqdm
from datetime import datetime
import pandas as pd
import itertools
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from multiprocessing.dummy import Pool as ThreadPool

print (datetime.now().strftime("%H:%M:%S start"))



00:17:48 start


In [3]:
import torch
from collections import Counter

domens_num = 4

urls_tokens = [{} for i in range(domens_num)]

with open('./urls.csv') as f_in:
    for line in tqdm(f_in):
        key, tokens = line.strip().split(',')
        splited = tokens.split('/')
        key = int(key)
        for i in range(domens_num):
            urls_tokens[i][key] = " ".join(splited[:i + 1])



libgomp: Invalid value for environment variable OMP_NUM_THREADS
14148535it [00:40, 347304.17it/s]


In [4]:
categories_num = 100000 #[100000, 100000, 100000, 100000]
urls_cat =  [{} for i in range(domens_num)]      
all_domens = []
for i in range(domens_num):
    all_domens.append(list(Counter(urls_tokens[i].values()).most_common()))
    all_domens_cat = {all_domens[-1][j][0] : (j + 1 if j < categories_num else categories_num + 1) for j in range(len(all_domens[-1]))}
    
    for key in urls_tokens[i].keys():
        urls_cat[i][key] = all_domens_cat[urls_tokens[i][key]]

In [120]:
for i in range(domens_num):
    print(all_domens[i][0], all_domens[i][10], all_domens[i][100], all_domens[i][1000], all_domens[i][10000], all_domens[i][100000])
    num_all = 0
    num_cut = 0
    for j in range(len(all_domens[i])):
        if j > 10000:
            num_cut += all_domens[i][j][1]
        num_all += all_domens[i][j][1]
    print("---------------------------")
    print(num_all, num_cut, num_all - num_cut, num_cut / num_all, (num_all - num_cut) / num_all) 
    print(all_domens[i][0][1] / num_all, all_domens[i][10][1] / num_all, all_domens[i][100][1] / num_all, all_domens[i][1000][1] / num_all, 
          all_domens[i][5000][1] / num_all, all_domens[i][10000][1] / num_all)
    print("---------------------------")

('ed95a9a5be30e4c8', 3466117) ('999fd0543f2499ba', 85182) ('e6f885f2c422c0f2', 12120) ('bf3ce51c12767354', 1095) ('5142755da7d57991', 70) ('2f080c94fcbaa428', 2)
---------------------------
14148535 1424776 12723759 0.10070130935817737 0.8992986906418227
0.24498062873647342 0.006020552657925361 0.0008566257919989597 7.73931718018862e-05 1.173266348777453e-05 4.947508699663958e-06
---------------------------
('ed95a9a5be30e4c8 5162fc6a223f248d', 533525) ('ed95a9a5be30e4c8 8a8c5028a7feb15', 62911) ('ed95a9a5be30e4c8 60512d6d9f8eac9', 12053) ('d7953daa3ea1ac1 527ad573b150ae33', 1186) ('fff96a2e31ce5220 71b78330059efc0a', 65) ('eaabd378883b49ab 70e13026148c016', 3)
---------------------------
14148535 5240748 8907787 0.3704092331820927 0.6295907668179073
0.03770885112840305 0.004446467425779418 0.0008518903193864241 8.382493311144934e-05 1.0884519139260708e-05 4.594115221116533e-06
---------------------------
('dcb69d5b9ce0d93 de6dc30c32ad79ae f11039529d498ce4', 132036) ('ed95a9a5be30e4c8 

In [163]:
print([len(all_domens[i]) for i in range(len(all_domens))])

[427915, 4157829, 8126039, 12744146]


In [84]:
for i in range(domens_num):
    print(max(urls_cat[i].values()))

1001
1001
1001
1001


In [85]:
print(max(all_domens_cat.values()))

1001


In [5]:
import torch
from torch import LongTensor

facts = [{} for i in range(domens_num)]
times = {}
with open('./facts.json') as f_in:
    for line in tqdm(f_in):
        j = json.loads(line.strip())
        uid = j.get('uid')
        for i in range(domens_num):
            facts[i][uid] = [urls_cat[i][x['fid']] for x in j.get('facts')]
        times[uid] = [x['ts'] for x in j.get('facts')]
print(list(times.items())[:10])
print(list(facts[0].items())[:10])
print(list(facts[1].items())[:10])
print(list(facts[2].items())[:10])
print(list(facts[3].items())[:10])

339405it [01:36, 3521.63it/s]


[('59e3393261202d419e3c2721a6e15f9f', [1464769462076, 1464769312076, 1464769282076, 1464769248691, 1464769068691, 1464768967747, 1464768953582, 1464768907747, 1464768787747, 1464764699151, 1464764639151, 1464764633759, 1464764609151, 1464764519151, 1464764384950, 1464764339151, 1464764279151, 1464764099151, 1464764030849, 1464763979151, 1464762899151, 1464762779151, 1464762719151, 1464762599151, 1464762569151, 1464762479151, 1464762449151, 1464762389151, 1464762299151, 1464762239151, 1464762199171, 1464762193661, 1464762191026, 1464762169171, 1464762138231, 1464762124110, 1464762059151, 1464761939151, 1464761909151, 1464761879151, 1464761789151, 1464761774841, 1464761665071, 1464761545071, 1464761275071, 1464761245071, 1464761125071, 1464761065071, 1464760975071, 1464760915071, 1464760885071, 1464760705071, 1464760610921, 1464760585071, 1464760559151, 1464760529151, 1464760519911, 1464760483771, 1464760478145, 1464760447815, 1464759957908, 1464759923649, 1464759921103, 1464759913447, 1

In [6]:
users_in_train = set()
#with open('./facts.txt','w') as f_out:
train_col1 = []
train_col2 = []
with open('./train.csv') as f_in:
    for line in tqdm(f_in):
        user1,user2 = line.strip().split(',')
        train_col1.append(user1)
        train_col2.append(user2)
        #f_out.write("%s %s\n" % (facts[user1], facts[user2]))
        users_in_train.update([user1,user2])
#users_for_predict = set(facts[0].keys()).difference(users_in_train)

#users_for_predict_list = sorted(list(users_for_predict))
"""
with open('./facts_test.tsv','w') as f_out:
    for x in users_for_predict_list:
        f_out.write("%s\t%s\n" % (x, facts[x]))
"""


506136it [00:00, 1149476.81it/s]


'\nwith open(\'./facts_test.tsv\',\'w\') as f_out:\n    for x in users_for_predict_list:\n        f_out.write("%s\t%s\n" % (x, facts[x]))\n'

In [7]:
users_in_val = set()
#with open('./facts.txt','w') as f_out:
val_col1 = []
val_col2 = []

with open('./competition-11171-1-data-competition-11171-1-data-reference_1/valid.csv') as f_in:
    for line in tqdm(f_in):
        user1,user2 = line.strip().split(',')
        val_col1.append(user1)
        val_col2.append(user2)
        #f_out.write("%s %s\n" % (facts[user1], facts[user2]))
        users_in_val.update([user1,user2])

print(users_in_val.intersection(users_in_train))



107654it [00:00, 1005107.76it/s]

set()


In [89]:
print(user1, user2)

6028350a3370ee7b0dcd2fa84fb5af63 c13737e4a082a34e495b85b5d0ed5809


In [90]:
print(list(facts[0].keys())[:30])

['59e3393261202d419e3c2721a6e15f9f', '00cb715802c694b13d23da9db609031c', '5f4730c95313408d82477f56d366241f', 'c78c7844e9d9a9c6368734a2b5c6c5b6', 'c470f1ef60a3f2fe2af01274dee5285e', '1be8b3b42fa5928ca890ed8979617231', '904a2838b5e63c96b198e7333246220b', 'ba75a6c11fd4047b9160937d7ff6321c', '89b28fc1aab17cd06bbe5e0c7fa7baae', 'd6f70c03851adf05412deda397a8d129', 'd04c6e0ba78b4adabb2d0190b65f07ed', 'd744b1343d58d35ff41337e3dbd1104f', '67cf0df56a2e26d9b31c88d813b5797d', 'bd4668164691dd73d977e7b553c1b9ac', 'bf125439bd889988ffda0ec94055caad', 'b6d4f66b93d42f7f9ffcbd01eb261d31', '5b640eb59bfa839380f14a04bca130fa', 'f62f27cb536d1f1ced1e789bc78341e1', '0fdd015ff14f12af51e415acaed0723f', 'd663dd9aa363e26b0eb69f0da506783f', '66557a33adbd722ae0b48aed77f2a57b', '885c093051b246d39751a9afb315fbd1', 'c1bbb323ec3f7c04e836f456e8e54a55', 'e49de048128d157d83c487d71047a5a7', 'af2cb3e5f5958410b946e51663f3e1a4', 'a145138d6c4d56acc6b788627cf7f2e0', '47ef3bda8e3816b0d1800fe8a623ac40', 'd6d97a12e704aea3e7cfb768d6

In [8]:
data = pd.DataFrame({"time":pd.Series(times), "url0": pd.Series(facts[0]), 
                      "url1": pd.Series(facts[1]),
                     "url2": pd.Series(facts[2]), 
                      "url3": pd.Series(facts[3])})

In [179]:
data['uid'] = data.index.values
import dask.dataframe as da
data_dask = da.from_pandas(data, chunksize=50000)
import pyarrow as pa
schema = pa.schema([
    ('uid', pa.string()),
    ('time', pa.list_(pa.int64())),
    ('url0', pa.list_(pa.int64())),
    ('url1', pa.list_(pa.int64())),
    ('url2', pa.list_(pa.int64())),
    ('url3', pa.list_(pa.int64()))
])

data_dask.to_parquet('~/shestov/src/matching/cikm2016_all_data_no_pairs_100000/', schema=schema)

In [9]:
def make_data(data, col1, col2):
    data1 = data.loc[np.array(col1), :]
    data2 = data.loc[np.array(col2), :]
    data1 = data1.rename(columns={"time": "data1_time", "url0": "data1_url0", 
                          "url1": "data1_url1",
                         "url2": "data1_url2", 
                          "url3": "data1_url3"})
    data2 = data2.rename(columns={"time": "data2_time", "url0": "data2_url0", 
                          "url1": "data2_url1",
                         "url2": "data2_url2", 
                          "url3": "data2_url3"})
    #test_data = data.loc[np.array(users_for_predict), :]
    return pd.concat([data1.reset_index(drop=True, inplace=False), data2.reset_index(drop=True, inplace=False)], axis=1)

In [128]:
train_data = make_data(data, train_col1, train_col2)
print(train_data)

                                               data1_time  \
0       [1465275422109, 1465275345560, 1465275305897, ...   
1       [1465275422109, 1465275345560, 1465275305897, ...   
2       [1465887404194, 1465887393619, 1465887390791, ...   
3       [1466063385004, 1466057017523, 1466053730311, ...   
4       [1465535145004, 1465478835004, 1465466532386, ...   
...                                                   ...   
506131  [1465539418786, 1465539389215, 1465539388786, ...   
506132  [1465539418786, 1465539389215, 1465539388786, ...   
506133  [1466284477814, 1466281994388, 1466281992950, ...   
506134  [1466269288826, 1465455036762, 1465455030347, ...   
506135  [1466284477814, 1466281994388, 1466281992950, ...   

                                               data1_url0  \
0       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
1       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
2       [469, 469, 469, 469, 469, 469, 469, 469, 469, ...   
3       [37, 931, 37, 3

In [161]:
def len_stats(data):
    all_len = []
    for i in range(data.shape[0]):
        all_len.append(len(data.values[i, 0]))

    print(sum(all_len) / data.shape[0])
    print(sorted(all_len)[data.shape[0] // 2])

len_stats(train_data)
len_stats(val_data)


171.85075157665133
98
168.49873669348096
97


In [129]:
val_data = make_data(data, val_col1, val_col2)
print(val_data)


                                               data1_time  \
0       [1464602620920, 1464602588917, 1464602560920, ...   
1       [1464864091690, 1464864004717, 1464794014717, ...   
2       [1464602620920, 1464602588917, 1464602560920, ...   
3       [1465655740830, 1465468287354, 1465468161943, ...   
4       [1465655740830, 1465468287354, 1465468161943, ...   
...                                                   ...   
107649  [1465882789586, 1465644523395, 1465643743395, ...   
107650  [1465636586003, 1465636454650, 1465636424650, ...   
107651  [1465296319274, 1465296316814, 1465296280738, ...   
107652  [1465296319274, 1465296316814, 1465296280738, ...   
107653  [1466173815004, 1466173305004, 1466167644973, ...   

                                               data1_url0  \
0       [1, 1, 1, 1, 1, 1, 1, 1, 15321, 1, 1, 2553, 49...   
1       [1, 1, 1, 1, 1, 6030, 49648, 87469, 693, 693, ...   
2       [1, 1, 1, 1, 1, 1, 1, 1, 15321, 1, 1, 2553, 49...   
3       [48, 8293, 2141

In [70]:
import dask.dataframe as da
import pyarrow as pa
def data_to_parquet(data, name):

    data_dask = da.from_pandas(data, chunksize=50000)
    
    schema = pa.schema([
        ('data1_time', pa.list_(pa.int64())),
        ('data1_url0', pa.list_(pa.int64())),
        ('data1_url1', pa.list_(pa.int64())),
        ('data1_url2', pa.list_(pa.int64())),
        ('data1_url3', pa.list_(pa.int64())),
        ('data2_time', pa.list_(pa.int64())),
        ('data2_url0', pa.list_(pa.int64())),
        ('data2_url1', pa.list_(pa.int64())),
        ('data2_url2', pa.list_(pa.int64())),
        ('data2_url3', pa.list_(pa.int64()))
    ])
    data_dask.to_parquet(name, schema=schema)

In [130]:
import dask.dataframe as da
train_data_dask = da.from_pandas(train_data, chunksize=50000)
import pyarrow as pa
schema = pa.schema([
    ('data1_time', pa.list_(pa.int64())),
    ('data1_url0', pa.list_(pa.int64())),
    ('data1_url1', pa.list_(pa.int64())),
    ('data1_url2', pa.list_(pa.int64())),
    ('data1_url3', pa.list_(pa.int64())),
    ('data2_time', pa.list_(pa.int64())),
    ('data2_url0', pa.list_(pa.int64())),
    ('data2_url1', pa.list_(pa.int64())),
    ('data2_url2', pa.list_(pa.int64())),
    ('data2_url3', pa.list_(pa.int64()))
])
train_data_dask.to_parquet('~/shestov/src/matching/cikm2016_data_train_100000/', schema=schema)
#dummy_data_val = da.from_pandas(dummy_data_val, chunksize=500)
#dummy_data_val.to_parquet('~/shestov/src/matching/dummy_data_val', schema=schema)


In [131]:
data_to_parquet(val_data, '~/shestov/src/matching/cikm2016_data_val_100000/')

In [105]:
print(val_data.shape, train_data.shape)

(107654, 10) (506136, 10)


In [157]:
print(val_without_pairs)

                                                                               time  \
c1b420279ea4352e72c68c1560e98a2d  [1465805819232, 1465805705524, 1465755159173, ...   
241f0478d7192d72d81fd55889d0d63d  [1464464228336, 1464373347249, 1464373273729, ...   
ce4d8356a328b9a7afac64e577071efa  [1466080126911, 1466063861374, 1466063531374, ...   
615b0fc5bd02f536ba7e60950d9d208f  [1466529514325, 1466445394325, 1466411284325, ...   
a28a4e24aa19aab794920afb2742d758  [1464533676763, 1464268094039, 1464254937850, ...   
...                                                                             ...   
2909717eecbabc456b7f6af73340fff9  [1464206053729, 1464129314065, 1464129296542, ...   
d64fadd04b0e33a6588db1f107b3837d  [1464199165700, 1464198775700, 1464198751048, ...   
3a75a61acab9c941201dae2c3058aaa2  [1466059861170, 1466000672310, 1465996860971, ...   
090f26239de343da15022c0307757e75  [1464795511555, 1464607549897, 1464602274315, ...   
ac35591497add1447f4a092c74f17a59  [14647670

In [159]:
print(uid_to_num)
print(num_to_uid)

000036f9a0d4284e72f48769fd99e660        0
0000efc524543b6c2c873f48e39c482b        1
0002123264d3d1f2d630a46f77337458        2
0002ae836f54fd505ffb068ae9185910        3
000354e2b34dde6b6a2b6d640098a96c        4
                                    ...  
fffc53c6d2d422495f71cfad224ee3ea    50141
fffccd0ad62f598d2407479a160bfff5    50142
fffd075ff096c7f587566ade683b8fe7    50143
fffd972aac10d7dc3fe37808d6873dfa    50144
fffdb30c520c88a4c129edcddfde21bf    50145
Length: 50146, dtype: int64
0        000036f9a0d4284e72f48769fd99e660
1        0000efc524543b6c2c873f48e39c482b
2        0002123264d3d1f2d630a46f77337458
3        0002ae836f54fd505ffb068ae9185910
4        000354e2b34dde6b6a2b6d640098a96c
                       ...               
50141    fffc53c6d2d422495f71cfad224ee3ea
50142    fffccd0ad62f598d2407479a160bfff5
50143    fffd075ff096c7f587566ade683b8fe7
50144    fffd972aac10d7dc3fe37808d6873dfa
50145    fffdb30c520c88a4c129edcddfde21bf
Length: 50146, dtype: object


In [11]:
val_without_pairs = data.loc[users_in_val, :]
val_without_pairs = val_without_pairs.rename(columns={"time": "data_time", "url0": "data_url0", 
                          "url1": "data_url1",
                         "url2": "data_url2", 
                          "url3": "data_url3"})
val_without_pairs['uid'] = val_without_pairs.index.values
import dask.dataframe as da
val_without_pairs_dask = da.from_pandas(val_without_pairs, chunksize=50000)
import pyarrow as pa
schema = pa.schema([
    ('uid', pa.string()),
    ('data_time', pa.list_(pa.int64())),
    ('data_url0', pa.list_(pa.int64())),
    ('data_url1', pa.list_(pa.int64())),
    ('data_url2', pa.list_(pa.int64())),
    ('data_url3', pa.list_(pa.int64()))
])
val_without_pairs_dask.to_parquet('~/shestov/src/matching/cikm2016_data_val_no_pairs_100000/', schema=schema)

/tmp/ipykernel_1361172/3982312907.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  val_without_pairs = data.loc[users_in_val, :]


In [12]:
print(val_without_pairs)

                                                                          data_time  \
cf7e44bd15617c23683b48c0325d692d  [1464091166212, 1464091151344, 1464091131877, ...   
2b4abca753dc01079a6ccc061e6293dd  [1466509996843, 1466509992416, 1465715336692, ...   
ba00c76f4bf6c443d313471a7a1b7068  [1466059672545, 1466059656296, 1466059654110, ...   
c0903567935d0b184884693eca1e3c36  [1465449098194, 1465449064391, 1465449056296, ...   
7dad5db55815d3ca1d3f43a6bee77b5f  [1466503440217, 1466503381439, 1466503380217, ...   
...                                                                             ...   
82de111518bc5e1cdbb29c728b48c857  [1463930138447, 1463930108447, 1463930048447, ...   
303a52e9058ffe8f0431a2bea21b2c4a  [1464382746204, 1464382740147, 1464382740144, ...   
186acffc3e659549600a66ebebba55b9  [1465328217251, 1465226084152, 1465225244152, ...   
5b91850431c87bfd3a3698bbce7e88b0  [1465371590357, 1465313835128, 1465218839960, ...   
3ac2464048eacc15b952eba911a170c6  [14666001

In [ ]:
train_without_pairs = data.loc[users_in_train, :]
train_without_pairs = train_without_pairs.rename(columns={"time": "data_time", "url0": "data_url0", 
                          "url1": "data_url1",
                         "url2": "data_url2", 
                          "url3": "data_url3"})
train_without_pairs['uid'] = train_without_pairs.index.values
import dask.dataframe as da
train_without_pairs_dask = da.from_pandas(train_without_pairs, chunksize=50000)
import pyarrow as pa
schema = pa.schema([
    ('uid', pa.string()),
    ('data_time', pa.list_(pa.int64())),
    ('data_url0', pa.list_(pa.int64())),
    ('data_url1', pa.list_(pa.int64())),
    ('data_url2', pa.list_(pa.int64())),
    ('data_url3', pa.list_(pa.int64()))
])
train_without_pairs_dask.to_parquet('~/shestov/src/matching/cikm2016_data_train_no_pairs_100000/', schema=schema)

In [ ]:
print(train_without_pairs_dask)

In [ ]:
print(val_without_pairs.sort_index())

In [167]:
val_without_pairs_left = val_without_pairs.rename(columns={"time": "data1_time", "url0": "data1_url0", 
                          "url1": "data1_url1",
                         "url2": "data1_url2", 
                          "url3": "data1_url3"})
val_without_pairs_left['uid'] = val_without_pairs_left.index.values
val_without_pairs_right = val_without_pairs.rename(columns={"time": "data2_time", "url0": "data2_url0", 
                          "url1": "data2_url1",
                         "url2": "data2_url2", 
                          "url3": "data2_url3"}).drop(columns=['uid'])
    #test_data = data.loc[np.array(users_for_predict), :]
val_without_pairs_m3 = pd.concat([val_without_pairs_left.reset_index(drop=True, inplace=False), 
                                  val_without_pairs_right.reset_index(drop=True, inplace=False)], axis=1)
val_without_pairs_m3_dask = da.from_pandas(val_without_pairs_m3, chunksize=50000)
schema = pa.schema([
    ('uid', pa.string()),
    ('data1_time', pa.list_(pa.int64())),
    ('data1_url0', pa.list_(pa.int64())),
    ('data1_url1', pa.list_(pa.int64())),
    ('data1_url2', pa.list_(pa.int64())),
    ('data1_url3', pa.list_(pa.int64())),
    ('data2_time', pa.list_(pa.int64())),
    ('data2_url0', pa.list_(pa.int64())),
    ('data2_url1', pa.list_(pa.int64())),
    ('data2_url2', pa.list_(pa.int64())),
    ('data2_url3', pa.list_(pa.int64()))
])
val_without_pairs_m3_dask.to_parquet('~/shestov/src/matching/cikm2016_data_val_no_pairs_true_100000_m3/', schema=schema)

In [168]:
print(val_without_pairs_m3)

                                              data1_time  \
0      [1465805819232, 1465805705524, 1465755159173, ...   
1      [1464464228336, 1464373347249, 1464373273729, ...   
2      [1466080126911, 1466063861374, 1466063531374, ...   
3      [1466529514325, 1466445394325, 1466411284325, ...   
4      [1464533676763, 1464268094039, 1464254937850, ...   
...                                                  ...   
50141  [1464206053729, 1464129314065, 1464129296542, ...   
50142  [1464199165700, 1464198775700, 1464198751048, ...   
50143  [1466059861170, 1466000672310, 1465996860971, ...   
50144  [1464795511555, 1464607549897, 1464602274315, ...   
50145  [1464767061479, 1464766041540, 1464766041479, ...   

                                              data1_url0  \
0      [1, 1, 1, 1, 1, 1, 1, 1, 46, 46, 46, 46, 46, 4...   
1      [17689, 135, 135, 135, 135, 135, 468, 468, 496...   
2      [96712, 103, 103, 103, 103, 103, 103, 103, 103...   
3      [7742, 7742, 7742, 7742, 7742, 7

In [155]:
print(val_without_pairs_dask)

Dask DataFrame Structure:
                                    time    url0    url1    url2    url3    uid
npartitions=2                                                                  
000036f9a0d4284e72f48769fd99e660  object  object  object  object  object  int64
ff3474944b2488dd4c530cc6967aa6b5     ...     ...     ...     ...     ...    ...
fffdb30c520c88a4c129edcddfde21bf     ...     ...     ...     ...     ...    ...
Dask Name: from_pandas, 1 graph layer


In [138]:
print(data)

                                                                               time  \
59e3393261202d419e3c2721a6e15f9f  [1464769462076, 1464769312076, 1464769282076, ...   
00cb715802c694b13d23da9db609031c  [1466522128380, 1466438202185, 1466419235273, ...   
5f4730c95313408d82477f56d366241f  [1465368385868, 1465368371220, 1465368367556, ...   
c78c7844e9d9a9c6368734a2b5c6c5b6  [1466058475146, 1466058464841, 1466058460134, ...   
c470f1ef60a3f2fe2af01274dee5285e  [1464792609823, 1464792579823, 1464786639823, ...   
...                                                                             ...   
d43e3701c4c6c1c496bbbf13a3c7d9af  [1464870611142, 1464870597634, 1464765643295, ...   
eba228dbf894c8da8413a56d65989aa0  [1465995415657, 1465995385657, 1465995346439, ...   
e53a9e875e3d54bee48c076d7c0b7f2c  [1466415584689, 1465837834982, 1465816619644, ...   
011701f6f75c437003c3d2fef13eaaf8  [1466348228144, 1466348198144, 1465673438021, ...   
70b43158faa35a7ce28eb84029186589  [14640197

In [ ]:
from functools import partial
from ptls.nn import TrxEncoder, RnnSeqEncoder
from ptls.frames.coles import CoLESModule

trx_encoder_params = dict(
    embeddings_noise=0.003,
    numeric_values={'amount_rur': 'identity'},
    embeddings={
        'trans_date': {'in': 800, 'out': 16},
        'small_group': {'in': 250, 'out': 16},
    },
)

seq_encoder = RnnSeqEncoder(
    trx_encoder=TrxEncoder(**trx_encoder_params),
    hidden_size=256,
    type='gru',
)

model = CoLESModule(
    seq_encoder=seq_encoder,
    optimizer_partial=partial(torch.optim.Adam, lr=0.001),
    lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=30, gamma=0.9),
)


In [4]:
print(list(urls_tokens.items())[:10])

[('9140201', 'ed95a9a5be30e4c8'), ('6684120', 'ed95a9a5be30e4c8'), ('9004649', 'ed95a9a5be30e4c8'), ('1414226', 'ed95a9a5be30e4c8'), ('4225543', 'ed95a9a5be30e4c8'), ('8141140', 'ed95a9a5be30e4c8'), ('13697961', 'ed95a9a5be30e4c8'), ('11292447', 'ed95a9a5be30e4c8'), ('12853589', '93bbf66ff7f27f7f'), ('2611126', '93bbf66ff7f27f7f')]


In [5]:
print(list(facts.items())[:10])

[('59e3393261202d419e3c2721a6e15f9f', 'ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 93bbf66ff7f27f7f 93bbf66ff7f27f7f 93bbf66ff7f27f7f 93bbf66ff7f27f7f ed95a9a5be30e4c8 a997482113271d8f ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 2f38166a9f476d14 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5be30e4c8 ed95a9a5b

In [ ]:
print datetime.now().strftime("%H:%M:%S done with preprocessing")
        
tf = TfidfVectorizer(lowercase=False, preprocessor=None).fit(map(lambda x: x.strip().split('\t')[-1], open('facts.txt').readlines()))

users, row_text = [], []
with open('facts_test.tsv') as f_in:
    for line in tqdm(f_in):
        user, t = line.strip().split('\t')
        users.append(user)
        row_text.append(t)
        
tf_test = tf.transform(row_text)

print datetime.now().strftime("%H:%M:%S done with tf-idf")

knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(tf_test, range(1, tf_test.shape[0] + 1))
pool = ThreadPool(16)

def get_predict(line):
    res = []
    user_id, tokens = line.strip().split('\t')
    cur_vect = tf.transform([tokens])
    tmp = knn.kneighbors(X=tf.transform([tokens]), n_neighbors=15, return_distance=True)
    for i in range(len(tmp[0][0])):
        if 0 < tmp[0][0][i] < 0.9999:
            if user_id < users[tmp[1][0][i]]:
                res.append((user_id, users[tmp[1][0][i]], tmp[0][0][i]))
            else:
                res.append((users[tmp[1][0][i]], user_id, tmp[0][0][i]))
    return res

lines = open('facts_test.tsv').readlines()
results = pool.map(get_predict, lines)

print datetime.now().strftime("%H:%M:%S done with predictions")

q = sorted(list(set([y for x in results for y in x])), key=lambda x: x[-1])

with open('submission.txt','w') as f_out:
    # there are 215,307 in the validation (phase 1) and test (phase 2) data sets
    # we use 50% of correct values for validation (phase 1) and 50% of correct values for test (phase 2)
    # you have to submit the same file for both phases and we will handle the calculation automatically
    for x in tqdm(q[:215307]):
        f_out.write("%s,%s\n" % (x[0], x[1]))


In [11]:
from ptls.preprocessing import PandasDataPreprocessor

preprocessor = PandasDataPreprocessor(
    col_id='client_id',
    col_event_time='trans_date',
    cols_category=['small_group'],
    cols_numerical=['amount_rur'],
)


libgomp: Invalid value for environment variable OMP_NUM_THREADS
